In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform

In [2]:
# Functions

# Get panel masks

def get_panel_masks(geojson_path, ref_img_path):
    '''This function generates a mask that leaves only pixels inside 
    vine panels unmasked. 
    
    Inputs:
    geojson_path (str): path to a geojson file with the panel geometries. 
    Must be in same CRS as image, otherwise un-comment code section starting with "dst_crs".
    ref_img_path (str): path to the .tif file for the image
    
    Outputs:
    A list of masks, as Boolean arrays. True values are masked pixels.
    False values are unmasked pixels.
    '''
    with open(geojson_path, 'r') as f:
        panels = json.load(f)
        features = panels['features']
        features_ordered = sorted(features, key= lambda feature:(feature['properties']['row'],feature['properties']['panel'])) # ground truth order.
        panel_aois = [f['geometry'] for f in features_ordered]
    
    with rio.open(ref_img_path) as src:
        dst_crs = src.crs
        
        # geojson features (the field block geometries)
        # are often in WGS84
        # project these to the image coordinates
        wgs84 = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
        proj_panel_aois = [transform(project, shape(p)) for p in panel_aois]

        masks = [raster_geometry_mask(src, [p], crop=False)[0]
                 for p in proj_panel_aois]                       
    return masks

# Get reflectance df

def get_rfl(img_path, masks):
    '''
    Function to extract per=panel reflectance values for 4-band SkySat images.
    Calculates the mean reflectance values for the pixels in each panel.
    The output is formatted as a pandas dataframe.
    
    Inputs:
    img_path - abs. path to the image       
    masks - a list of masks that correspond to the panel aois. 
    '''
    
    with rio.open(img_path,'r') as img:
        def _get_stats(mask, panel_number):
            panel_blue = np.ma.mean(np.ma.array(img.read(1), mask=mask))#/10000
            panel_green = np.ma.mean(np.ma.array(img.read(2), mask=mask))#/10000
            panel_red = np.ma.mean(np.ma.array(img.read(3), mask=mask))#/10000
            panel_nir = np.ma.mean(np.ma.array(img.read(4), mask=mask))#/10000
            veg_band= np.ma.mean(np.ma.array(img.read(5),mask=mask))
            savi_exp = np.ma.mean(np.ma.array(img.read(6),mask=mask))
            savi = np.ma.mean(np.ma.array(img.read(7),mask=mask))
            msavi = np.ma.mean(np.ma.array(img.read(8),mask=mask))
            evi = np.ma.mean(np.ma.array(img.read(9),mask=mask))
            arvi = np.ma.mean(np.ma.array(img.read(10),mask=mask))
            gri = np.ma.mean(np.ma.array(img.read(11),mask=mask))
            ndvi = np.ma.mean(np.ma.array(img.read(12),mask=mask))
            nir_blue = np.ma.mean(np.ma.array(img.read(13),mask=mask))
            nir_green = np.ma.mean(np.ma.array(img.read(14),mask=mask))
            red_blue = np.ma.mean(np.ma.array(img.read(15),mask=mask))
            green_red = np.ma.mean(np.ma.array(img.read(16),mask=mask))
            green_blue = np.ma.mean(np.ma.array(img.read(17),mask=mask))
            
            
            return {'acquired': img_path.split('/')[-1].split('_')[-6], # date from image filename
                    'plot': panel_number + 1,    #add one - panels are numbered starting at 1 in the 'real world'
                    'blue': panel_blue,
                    'green': panel_green,
                    'red': panel_red,
                    'nir': panel_nir,
                    'veg_smr': veg_band,
                    'savi_exp': savi_exp,
                    'savi': savi,
                    'msavi': msavi,
                    'evi': evi,
                    'arvi': arvi,
                    'gri': gri,
                    'ndvi': ndvi,
                    'nir_blue': nir_blue, 
                    'nir_green': nir_green,
                    'red_blue': red_blue,
                    'green_red':green_red,
                    'green_blue': green_blue
                   }
        data = [_get_stats(m, i) for i, m in enumerate(masks)]  # sets the parameters for the nested function as the mask (m) and corresponding panel number (i). 
        df = pd.DataFrame(data)
    
    return df

In [3]:
# image directory containing co-registered SkySat scenes 

img_dir = '../data/images/2021/SkySat/smr_vis_2021/'

# image pathnames
img_paths = glob.glob(img_dir+'*.tif')

# geojson with panel geometries

json_path = '../data/crit_panels_geojson.geojson'
#json_path
img_paths

['../data/images/2021/SkySat/smr_vis_2021/SMR_allVIs_20210707_150122_ssc19d2_0016_ar_clipped.tif',
 '../data/images/2021/SkySat/smr_vis_2021/SMR_allVIs_20210726_160338_ssc1d2_0013_ar_clipped.tif',
 '../data/images/2021/SkySat/smr_vis_2021/SMR_allVIs_20210802_184019_ssc10d2_0005_ar_clipped.tif',
 '../data/images/2021/SkySat/smr_vis_2021/SMR_allVIs_20210809_185329_ssc9d2_0014_ar_clipped.tif',
 '../data/images/2021/SkySat/smr_vis_2021/SMR_allVIs_20210816_171007_ssc18d2_0015_ar_clipped.tif']

In [4]:
img_paths[0].split('/')[-1].split('_')[-6]

'20210707'

In [5]:
# Get dataframes for all images in directory

master_df = pd.DataFrame()

for i in img_paths:
    masks = get_panel_masks(json_path, i)
    df = get_rfl(i, masks)
    master_df = pd.concat([master_df, df], axis=0)
    master_df = master_df.sort_values(by=['acquired', 'plot'])

master_df

,acquired,plot,blue,green,red,nir,veg_smr,savi_exp,savi,msavi,evi,arvi,gri,ndvi,nir_blue,nir_green,red_blue,green_red,green_blue
0,20210707,1,0.032006,0.066432,0.040084,0.290003,0.578012,0.473011,0.451348,0.251582,0.484028,0.714474,1.663994,0.756461,0.800678,0.626750,0.111632,0.248109,0.350125
1,20210707,2,0.033528,0.067879,0.043324,0.270290,0.519652,0.423616,0.418255,0.208013,0.443640,0.671035,1.572952,0.723158,0.778815,0.598146,0.127225,0.221649,0.339256
2,20210707,3,0.033903,0.067897,0.042947,0.273817,0.528627,0.431190,0.423939,0.215251,0.451946,0.680747,1.584797,0.728672,0.779568,0.602466,0.117626,0.225604,0.334337
3,20210707,4,0.032868,0.067358,0.042055,0.280455,0.548418,0.447186,0.434635,0.229500,0.463358,0.690756,1.603778,0.738937,0.790084,0.612390,0.122486,0.231461,0.344171
4,20210707,5,0.032104,0.066636,0.038836,0.292664,0.587151,0.481896,0.457826,0.258864,0.493827,0.730582,1.718833,0.765581,0.802129,0.628947,0.095024,0.263758,0.349998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,20210816,316,0.011672,0.062224,0.041776,0.253272,0.513864,0.403711,0.399060,0.178841,0.373333,0.558840,1.518394,0.717312,0.912213,0.605556,0.590428,0.202623,0.700933
316,20210816,317,0.007941,0.059369,0.035369,0.256410,0.545402,0.431291,0.418762,0.195855,0.392190,0.606819,1.692738,0.757778,0.940200,0.624041,0.650998,0.255787,0.772210
317,20210816,318,0.009167,0.060350,0.038300,0.249923,0.519761,0.408312,0.402772,0.178483,0.374988,0.575653,1.593335,0.734620,0.929467,0.611113,0.631341,0.226821,0.745786
318,20210816,319,0.006245,0.058310,0.034372,0.253779,0.544837,0.429847,0.417589,0.192616,0.388196,0.605127,1.706633,0.761645,0.952218,0.626496,0.705967,0.259840,0.812638


In [6]:
# Append corresponding row and panel numbers for each plot 

panel_csv = pd.read_csv('../data/crit_panels.csv')
cols = panel_csv.iloc[:,1:] # adjust this according to the .csv
#cols
appended = pd.concat([master_df, cols],axis=1)
appended

,acquired,plot,blue,green,red,nir,veg_smr,savi_exp,savi,msavi,...,arvi,gri,ndvi,nir_blue,nir_green,red_blue,green_red,green_blue,row,panel
0,20210707,1,0.032006,0.066432,0.040084,0.290003,0.578012,0.473011,0.451348,0.251582,...,0.714474,1.663994,0.756461,0.800678,0.626750,0.111632,0.248109,0.350125,1,1
1,20210707,2,0.033528,0.067879,0.043324,0.270290,0.519652,0.423616,0.418255,0.208013,...,0.671035,1.572952,0.723158,0.778815,0.598146,0.127225,0.221649,0.339256,1,2
2,20210707,3,0.033903,0.067897,0.042947,0.273817,0.528627,0.431190,0.423939,0.215251,...,0.680747,1.584797,0.728672,0.779568,0.602466,0.117626,0.225604,0.334337,1,3
3,20210707,4,0.032868,0.067358,0.042055,0.280455,0.548418,0.447186,0.434635,0.229500,...,0.690756,1.603778,0.738937,0.790084,0.612390,0.122486,0.231461,0.344171,1,4
4,20210707,5,0.032104,0.066636,0.038836,0.292664,0.587151,0.481896,0.457826,0.258864,...,0.730582,1.718833,0.765581,0.802129,0.628947,0.095024,0.263758,0.349998,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,20210816,316,0.011672,0.062224,0.041776,0.253272,0.513864,0.403711,0.399060,0.178841,...,0.558840,1.518394,0.717312,0.912213,0.605556,0.590428,0.202623,0.700933,20,12
316,20210816,317,0.007941,0.059369,0.035369,0.256410,0.545402,0.431291,0.418762,0.195855,...,0.606819,1.692738,0.757778,0.940200,0.624041,0.650998,0.255787,0.772210,20,13
317,20210816,318,0.009167,0.060350,0.038300,0.249923,0.519761,0.408312,0.402772,0.178483,...,0.575653,1.593335,0.734620,0.929467,0.611113,0.631341,0.226821,0.745786,20,14
318,20210816,319,0.006245,0.058310,0.034372,0.253779,0.544837,0.429847,0.417589,0.192616,...,0.605127,1.706633,0.761645,0.952218,0.626496,0.705967,0.259840,0.812638,20,15


In [7]:
appended.acquired.unique()


array(['20210707', '20210726', '20210802', '20210809', '20210816'],
      dtype=object)

In [8]:
scout_csv = pd.read_csv('../data/scout/scout_2021_gdf.csv')
scout_csv.Date.unique()

array(['2021-06-24', '2021-07-01', '2021-07-07', '2021-07-15',
       '2021-07-27', '2021-08-03', '2021-08-10', '2021-08-17'],
      dtype=object)

In [9]:
# Match scout data to image data

# Define filtering and matching parameters

first_row = 1
last_row = 20

match_dates = ['2021-07-07','2021-07-27', '2021-08-03', '2021-08-10', '2021-08-17']
match_cols =['row', 'panel', 'Date']


# Filter and join datasets

img_filtered = appended[appended['row']>=first_row]
img_filtered = img_filtered[img_filtered['row']<=last_row]

scout_filtered = scout_csv[scout_csv['Row']>=first_row]
scout_filtered = scout_csv[scout_csv['Row']<=last_row]

scout_filtered = scout_filtered[scout_filtered['Date'].isin(match_dates)]
scout_filtered = scout_filtered.rename(columns={'Row':'row', 'Panel':'panel'})
scout_filtered[["row", "panel"]] = scout_filtered[["row", "panel"]].apply(pd.to_numeric)

# Create a 'Date' column in the image dataframe to match the scout date
def match_days(row):
    if row['acquired']=='20210707':
        return '2021-07-07'
    if row['acquired']=='20210726':
        return '2021-07-27'
    if row['acquired']=='20210802':
        return '2021-08-03'
    if row['acquired']=='20210809':
        return '2021-08-10'
    if row['acquired']=='20210816':
        return '2021-08-17'

img_filtered['Date'] = img_filtered.apply(lambda row: match_days(row), axis=1)

# Join and drop NANs + unneeded columns

joined = img_filtered.join(scout_filtered.set_index(match_cols), on=match_cols)

cleaned = joined.dropna(axis=0,subset=['DM_severity'])
cleaned.info()
## Uncomment below for masked rasters
#no_nullpix = cleaned.dropna(axis=0,subset=['blue', 'green', 'red', 'nir'])

#no_nullpix.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 0 to 310
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     1195 non-null   object 
 1   plot         1195 non-null   int64  
 2   blue         1195 non-null   float64
 3   green        1195 non-null   float64
 4   red          1195 non-null   float64
 5   nir          1195 non-null   float64
 6   veg_smr      1195 non-null   float64
 7   savi_exp     1195 non-null   float64
 8   savi         1195 non-null   float64
 9   msavi        1195 non-null   float64
 10  evi          1195 non-null   float64
 11  arvi         1195 non-null   float64
 12  gri          1195 non-null   float64
 13  ndvi         1195 non-null   float64
 14  nir_blue     1195 non-null   float64
 15  nir_green    1195 non-null   float64
 16  red_blue     1195 non-null   float64
 17  green_red    1195 non-null   float64
 18  green_blue   1195 non-null   float64
 19  row    

In [18]:
# Export dataframe as .csv

# output_fp = '../data/img_scout_dfs/2021/'

# cleaned.to_csv(output_fp+'smr_vis_skysat_scout_2021.csv', index=False)

In [10]:
# Merge incidence data

inc = pd.read_csv('/Users/kathleenkanaley/Desktop/grapes_from_space/data/scout/scout_incidence/scout_inc_2021.csv')

In [11]:
print(inc.columns)
print(cleaned.columns)

Index(['Date', 'Row', 'Panel', 'Treatment', 'Block', 'PM_severity',
       'DM_severity', 'total_dis', 'geometry', 'centroid', 'PM_inc', 'DM_inc'],
      dtype='object')
Index(['acquired', 'plot', 'blue', 'green', 'red', 'nir', 'veg_smr',
       'savi_exp', 'savi', 'msavi', 'evi', 'arvi', 'gri', 'ndvi', 'nir_blue',
       'nir_green', 'red_blue', 'green_red', 'green_blue', 'row', 'panel',
       'Date', 'Treatment', 'Block', 'PM_severity', 'DM_severity', 'total_dis',
       'geometry', 'centroid'],
      dtype='object')


In [12]:
inc.rename(columns={"Row": "row", "Panel": "panel"}, inplace=True)
inc.columns

Index(['Date', 'row', 'panel', 'Treatment', 'Block', 'PM_severity',
       'DM_severity', 'total_dis', 'geometry', 'centroid', 'PM_inc', 'DM_inc'],
      dtype='object')

In [13]:
merged_2021 = cleaned.merge(inc, how='left', on=['Date', 
                                               'row', 
                                               'panel',
                                               'Treatment', 
                                               'Block', 
                                               'PM_severity','DM_severity',
                                              'total_dis','geometry','centroid'])
merged_2021.shape


(1231, 31)

In [14]:
cleaned.shape

(1195, 29)

In [15]:
merged_2021

,acquired,plot,blue,green,red,nir,veg_smr,savi_exp,savi,msavi,...,Date,Treatment,Block,PM_severity,DM_severity,total_dis,geometry,centroid,PM_inc,DM_inc
0,20210707,1,0.032006,0.066432,0.040084,0.290003,0.578012,0.473011,0.451348,0.251582,...,2021-07-07,5.0,DM,0.00,0.00,0.05,POLYGON ((-77.0152645742787 42.878300949250196...,POINT (-77.0153084903025 42.8783140936145),0.0,0.0
1,20210707,2,0.033528,0.067879,0.043324,0.270290,0.519652,0.423616,0.418255,0.208013,...,2021-07-07,7.0,DM,0.00,0.00,0.10,POLYGON ((-77.01535545427274 42.87831851925121...,POINT (-77.0153965503027 42.87833067861654),0.0,0.0
2,20210707,3,0.033903,0.067897,0.042947,0.273817,0.528627,0.431190,0.423939,0.215251,...,2021-07-07,9.0,DM,0.00,0.00,0.10,POLYGON ((-77.01544069426711 42.87833411925217...,POINT (-77.01548247530272 42.87834674861851),0.0,0.0
3,20210707,4,0.032868,0.067358,0.042055,0.280455,0.548418,0.447186,0.434635,0.229500,...,2021-07-07,15.0,DM,0.00,0.00,0.60,POLYGON ((-77.01552730426143 42.87835065925314...,POINT (-77.01557007030289 42.87836309362055),0.0,0.0
4,20210707,5,0.032104,0.066636,0.038836,0.292664,0.587151,0.481896,0.457826,0.258864,...,2021-07-07,3.0,DM,0.00,0.00,0.10,POLYGON ((-77.01561588425558 42.87836680925413...,POINT (-77.01565649030296 42.878378758622546),0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,20210816,303,0.009360,0.059870,0.038270,0.248173,0.516014,0.404720,0.400398,0.175200,...,2021-08-17,18.0,PM,13.20,0.00,13.20,POLYGON ((-77.01536426393243 42.87876078918534...,POINT (-77.01540822030267 42.878773523616815),90.0,5.0
1227,20210816,305,0.015918,0.065997,0.046033,0.257685,0.504230,0.396905,0.394950,0.179668,...,2021-08-17,31.0,PM,0.90,0.00,0.90,POLYGON ((-77.01658187329902 42.87901510909438...,POINT (-77.01662629530372 42.8790284536449),20.0,0.0
1228,20210816,307,0.012376,0.061769,0.041824,0.254628,0.516814,0.405452,0.400873,0.181121,...,2021-08-17,40.0,PM,16.60,0.25,17.05,POLYGON ((-77.0164079733105 42.878982759092416...,POINT (-77.01645016030366 42.87899534864085),85.0,0.0
1229,20210816,309,0.016079,0.064390,0.045383,0.246759,0.481452,0.378002,0.381393,0.160394,...,2021-08-17,14.0,PM,0.00,0.00,0.05,POLYGON ((-77.01623108332213 42.87894828909042...,POINT (-77.01627422530342 42.878961353636775),0.0,0.0


In [16]:
merged_2021_nodup = merged_2021.drop_duplicates()
merged_2021_nodup.shape

(1195, 31)

In [17]:
cleaned.shape

(1195, 29)

In [18]:
output_fp = '/Users/kathleenkanaley/Desktop/grapes_from_space/data/img_scout_dfs/'
# merged_2021_nodup.to_csv(output_fp+'2021/smr_vis_skysat_scout_2021.csv', index=False)